In [2]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
import time

In [56]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9",
    "Referer": "https://www.google.com/"
}

In [57]:
url = 'https://markets.businessinsider.com/news/nvda-stock?p=1'

page = requests.get(url, headers= headers)

In [61]:
print(page.status_code)

503


In [58]:
soup = BeautifulSoup(page.text, 'lxml')

In [59]:
articles = soup.find_all('div', class_ ='latest-news__story')

In [60]:
articles

[]

In [7]:
# for row in articles:
#     print(row.text)
    

In [8]:
import pandas as pd


data = []
page_number = 1

while True:
    try:
        url = f'https://markets.businessinsider.com/news/nvda-stock?p={page_number}'

        page = requests.get(url)    

        soup = BeautifulSoup(page.text, 'lxml')
        articles = soup.find_all('div', class_ ='latest-news__story')

        if not articles:
            # Break the loop if no articles are found on the current page
            break

        for row in articles:
            datetime = row.find('time', class_= 'latest-news__date').get('datetime')
            title = row.find('a', class_= 'news-link').text
            source = row.find('span', class_= 'latest-news__source').text
            link = row.find('a',class_= 'news-link').get('href')
            data.append([datetime,title,source,link])
        
        page_number +=1
    
    except Exception as e:
        print(e)
        break



df = pd.DataFrame(data, columns = [ 'DateTime', 'Title', 'Source', 'Link'])

# df.to_csv('source.csv', index=False)


In [115]:
df= pd.read_csv("source.csv")

In [116]:
from transformers import pipeline

pipe = pipeline("text-classification", model="ProsusAI/finbert", framework="pt")

Device set to use cpu


In [117]:
def sentiment_analysis(text):
    data = pipe(text)
    return data


In [118]:
text = df['Title']

In [119]:
sentiment = []

for i in text:
        sentiment.append(sentiment_analysis(str(i)))

In [120]:
labels = [res[0]['label'] for res in sentiment]
scores = [res[0]['score'] for res in sentiment]

In [121]:
df['Label'] = labels
df['Score'] = scores

In [122]:
def sentiment_score(label):
    if label == 'positive':
        return 1
    if label == 'neutral':
        return 0
    if label == 'negative':
        return -1


In [123]:
df['Label'] = df['Label'].apply(sentiment_score)

In [124]:
df.to_csv('sentiment_score.csv', index = False)

In [19]:
# import pandas as pd 

# df = pd.read_csv('sentiment_score.csv')

In [20]:
# df.drop('Unnamed: 0', axis= 1, inplace=True)

In [125]:
df['Date'] = pd.to_datetime(df['DateTime'])
df['Date'] = df['Date'].dt.date

C:\Users\user\AppData\Local\Temp\ipykernel_1764\2354886005.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Date'] = pd.to_datetime(df['DateTime'])


In [126]:
print(df.duplicated().sum())

1


In [127]:
df.drop('DateTime', axis = 1, inplace=True)

In [128]:
df = df.set_index('Date')

In [131]:
def count_positive(value):

    if value == 1:
        return 1
    else:
        return 0

def count_neg(value):

    if value == -1:
        return 1
    else:
        return 0

def count_neu(value):

    if value == 0:
        return 1
    else:
        return 0


In [132]:
df['Positive'] = df['Label'].apply(count_positive)
df['Negative'] = df['Label'].apply(count_neg)
df['Neutral'] = df['Label'].apply(count_neu)

In [133]:
df_score = df.groupby(df.index).agg({
    'Score': 'mean',
    'Positive': 'sum',
    'Negative': 'sum',
    'Neutral': 'sum'
})

In [134]:
df_score['Total_Sentiment'] = df_score.apply(lambda row: row['Positive'] - row['Negative']/(row['Positive'] + row['Negative'] + row['Neutral']), axis=1)

In [135]:
df_score.tail()

,Score,Positive,Negative,Neutral,Total_Sentiment
Date,,,,,
2025-02-11,0.840833,4,2,16,3.909091
2025-02-12,0.788741,5,4,8,4.764706
2025-02-13,0.772654,10,3,9,9.863636
2025-02-14,0.801732,6,1,7,5.928571
2025-02-15,0.823377,0,1,1,-0.500000


In [136]:
print(df_score.duplicated().sum())

6


In [138]:
df_score.drop_duplicates(inplace=True)

In [ ]:
import mysql.connector as connector

In [159]:
try:
    conn = connector.connect(user='root', password='123456', host='localhost', port='3306', database='my_db')
except connector.Error as er:
    print('Error code:', er.errorno)
    print('Error msg:', er.msg)

mycursor = conn.cursor()

In [ ]:

mycursor.execute('SELECT * FROM stock_price')
price =  mycursor.fetchall()

In [ ]:
df_price = pd.DataFrame(price, columns=['Date', 'Open',	'High',	'Low',	'Close',	'Volume',	'Dividends',	'Stock Splits'])

In [ ]:
df_price.set_index('Date', drop=True)

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
1999-01-22,0.040122,0.044779,0.035585,0.037615,2.714690e+09,0.0,0.0
1999-01-25,0.040600,0.042033,0.037615,0.041556,5.104800e+08,0.0,0.0
1999-01-26,0.042033,0.042869,0.037734,0.038331,3.432000e+08,0.0,0.0
1999-01-27,0.038451,0.039406,0.036301,0.038212,2.443680e+08,0.0,0.0
1999-01-28,0.038212,0.038451,0.037854,0.038092,2.275200e+08,0.0,0.0
...,...,...,...,...,...,...,...
2025-02-14,136.480000,139.250000,135.500000,138.850000,1.954800e+08,0.0,0.0
2025-02-18,141.270000,143.440000,137.930000,139.400000,2.191770e+08,0.0,0.0
2025-02-19,139.510000,141.360000,137.220000,139.230000,1.675360e+08,0.0,0.0


In [141]:
print(df_price.duplicated().sum())

17210


In [142]:
df_price.drop_duplicates(inplace=True)

In [143]:
df_merge = df_price.merge(df_score, on = 'Date')

In [144]:
df_merge.set_index('Date', drop=True, inplace=True)

In [145]:
df_merge.to_csv('test.csv')

In [146]:
len(df_merge)

1909

In [151]:
print(df_merge.index.duplicated().sum())

316


In [153]:
df_merge = df_merge[~df_merge.index.duplicated()]

In [155]:
print(df_merge.index.duplicated().sum())

0


In [42]:
# mycursor.execute("CREATE TABLE price_sentiment (Date DATE, Open FLOAT(24), High FLOAT(24), Low FLOAT(24), Close FLOAT(24), Volume FLOAT(24), Dividends FLOAT(24), Stock_Splits FLOAT(24), Score FLOAT(24), Positive	INT, Negative INT,	Neutral INT, 	Total_Sentiment FLOAT(24))")

In [156]:
sql = "INSERT INTO  price_sentiment (Date, Open, High, Low, Close, Volume, Dividends, Stock_Splits, Score,	Positive,	Negative,	Neutral,	Total_Sentiment) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"

In [157]:
tuples_list = list(df_merge.itertuples(index=True, name=None))


In [160]:
mycursor.executemany(sql, tuples_list)

In [161]:
conn.commit()

mycursor.close()
conn.close()